# Fetching Daily data
Our goal in this notebook is to backfill the model with important historical features for Solana.

In [1]:
import requests
import pandas as pd
import numpy as np
import yfinance as yf
import hopsworks
import os
from datetime import datetime
from dotenv import load_dotenv

c:\Users\samyz\miniconda\envs\ca\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Doing the API requests 
- Get request

In [ ]:
ticker = "SOL-USD"
data = yf.download(ticker, start="2020-01-01", end="2025-12-15", interval="1d")

data.head()


[*********************100%***********************]  1 of 1 completed


Price,Close,High,Low,Open,Volume
Ticker,SOL-USD,SOL-USD,SOL-USD,SOL-USD,SOL-USD
Date,,,,,
2020-04-10,0.951054,1.313487,0.694187,0.832005,87364276
2020-04-11,0.776819,1.049073,0.765020,0.951054,43862444
2020-04-12,0.882507,0.956670,0.762426,0.785448,38736897
2020-04-13,0.777832,0.891603,0.773976,0.890760,18211285
2020-04-14,0.661925,0.796472,0.628169,0.777832,16747614


In [ ]:
data = data.reset_index()
data.columns = ['date', 'close', 'high', 'low', 'open', 'volume']

data = data[['date', 'close', 'high', 'low', 'open', 'volume']]

data.head()

✅ 2075 jours de données récupérés!
📅 Du 2020-04-10 00:00:00 au 2025-12-14 00:00:00


,date,close,high,low,open,volume
0,2020-04-10,0.951054,1.313487,0.694187,0.832005,87364276
1,2020-04-11,0.776819,1.049073,0.765020,0.951054,43862444
2,2020-04-12,0.882507,0.956670,0.762426,0.785448,38736897
3,2020-04-13,0.777832,0.891603,0.773976,0.890760,18211285
4,2020-04-14,0.661925,0.796472,0.628169,0.777832,16747614


# Extract and transform the data

In [ ]:
df = data.copy()


df.head()


✅ DataFrame 'df' créé avec 2075 lignes
📊 Colonnes: ['date', 'close', 'high', 'low', 'open', 'volume']


,date,close,high,low,open,volume
0,2020-04-10,0.951054,1.313487,0.694187,0.832005,87364276
1,2020-04-11,0.776819,1.049073,0.765020,0.951054,43862444
2,2020-04-12,0.882507,0.956670,0.762426,0.785448,38736897
3,2020-04-13,0.777832,0.891603,0.773976,0.890760,18211285
4,2020-04-14,0.661925,0.796472,0.628169,0.777832,16747614


# Computing Model Independant Features

## Moving averages for 7 days and 30 days

In [ ]:
df['close_7d_ma'] = df['close'].rolling(window=7).mean()
df['close_30d_ma'] = df['close'].rolling(window=30).mean()
df['close_50d_ma'] = df['close'].rolling(window=50).mean()
df['close_200d_ma'] = df['close'].rolling(window=200).mean()

df['volume_7d_ma'] = df['volume'].rolling(window=7).mean()
df['volume_30d_ma'] = df['volume'].rolling(window=30).mean()

df['ma_7_30_cross'] = np.where(df['close_7d_ma'] > df['close_30d_ma'], 1, 0)
df['ma_50_200_cross'] = np.where(df['close_50d_ma'] > df['close_200d_ma'], 1, 0)

df.tail(10)

✅ Moving averages computed!

📊 Preview with MA:


,date,close,high,low,open,volume,close_7d_ma,close_30d_ma,close_50d_ma,close_200d_ma,volume_7d_ma,volume_30d_ma,ma_7_30_cross,ma_50_200_cross
2065,2025-12-05,133.322998,140.379883,131.135300,139.013977,4557254539,136.037969,141.816481,159.285311,177.674701,4.658491e+09,5.505638e+09,0,0
2066,2025-12-06,132.348450,134.069839,131.506577,133.324371,2061286212,135.504646,141.059232,158.291584,177.494025,4.606056e+09,5.389562e+09,0,0
2067,2025-12-07,132.093567,136.232681,128.285110,132.349869,3824892147,135.294467,140.072078,157.180250,177.286084,4.706216e+09,5.298198e+09,0,0
2068,2025-12-08,133.278229,139.207352,131.674652,132.095993,4710407082,136.232925,139.244602,156.089850,177.054020,4.491672e+09,5.329303e+09,0,0
2069,2025-12-09,137.901611,144.742783,131.870728,133.278336,5869893316,136.122554,138.359528,155.052869,176.874401,4.448274e+09,5.373698e+09,0,0
2070,2025-12-10,136.469025,142.442352,135.862869,137.906097,5949930252,134.918344,137.329595,154.068924,176.676535,4.468354e+09,5.391217e+09,0,0
2071,2025-12-11,136.371567,137.678070,129.384018,136.463760,5833676784,134.540778,136.723037,153.193393,176.479765,4.686763e+09,5.378938e+09,0,0
2072,2025-12-12,132.324295,139.987320,130.679871,136.369934,5282060650,134.398106,136.025112,152.012113,176.266847,4.790307e+09,5.355684e+09,0,0
2073,2025-12-13,133.247467,134.245132,131.985809,132.324036,2024059188,134.526537,135.629745,150.805858,176.049008,4.784988e+09,5.177986e+09,0,0
2074,2025-12-14,129.481094,133.590988,128.901657,133.247620,2489001258,134.153327,135.323228,149.514672,175.835466,4.594147e+09,4.988601e+09,0,0


## RSI
- RSI measures momentum (0-100)
- RSI > 70 = overbought
- RSI < 30 = oversold

In [ ]:
def calculate_rsi(data, period = 14):
    """Calculate the Relative Strength Index (RSI) for a given DataFrame.
    
    Args:
        data (pd.Series): Series containing 'close' prices.
        period (int): The period over which to calculate RSI. Default is 14.

    Returns:
        pd.Series: A pandas Series containing the RSI values.
    """
    # Calculate price changes
    delta = data.diff()

    # Separate gains and losses
    gain = delta.where(delta > 0, 0)
    loss = -delta.where(delta < 0, 0)

    # Calculate average gain and loss
    avg_gain = gain.rolling(window=period, min_periods=period).mean()
    avg_loss = loss.rolling(window=period, min_periods=period).mean()

    # Calculate RS
    rs = avg_gain / avg_loss

    # Calculate RSI
    rsi = 100 - (100 / (1 + rs))

    return rsi

df['rsi_14'] = calculate_rsi(df['close'], period=14)

✅ RSI calculated!

📊 Preview with RSI:
           close     rsi_14
2070  136.469025  44.001300
2071  136.371567  45.735803
2072  132.324295  45.226336
2073  133.247467  47.309720
2074  129.481094  46.213522


## MACD (Moving Average Convergence Divergence)
- MACD Line = EMA12 - EMA26
- Signal Line = EMA9 of MACD
- Histogram = MACD - Signal

In [ ]:
def calculate_macd(data, fast=12, slow=26, signal=9):
    """Calculate the Moving Average Convergence Divergence (MACD) for a given DataFrame.
    
    Args:
        data (pd.Series): Series containing 'close' prices.
        fast (int): The period for the fast EMA. Default is 12.
        slow (int): The period for the slow EMA. Default is 26.
        signal (int): The period for the signal line EMA. Default is 9.
    
    Returns:
        Tuple of (macd_line, signal_line, histogram)
    """

    # Calculate the fast and slow EMAs
    ema_fast = data.ewm(span=fast, adjust=False).mean()
    ema_slow = data.ewm(span=slow, adjust=False).mean()

    # Calculate MACD line
    macd_line = ema_fast - ema_slow

    # Calculate signal line
    signal_line = macd_line.ewm(span=signal, adjust=False).mean()

    # Histogram = macd_line - signal_line
    histogram = macd_line - signal_line

    return macd_line, signal_line, histogram

df['macd'], df['macd_signal'], df['macd_histogram'] = calculate_macd(df['close'])


✅ MACD calculated!

📊 Preview with MACD:
           close      macd  macd_signal  macd_histogram
2065  133.322998 -5.985799    -7.978959        1.993160
2066  132.348450 -5.935256    -7.570218        1.634963
2067  132.093567 -5.848350    -7.225845        1.377494
2068  133.278229 -5.619111    -6.904498        1.285387
2069  137.901611 -5.006656    -6.524930        1.518274
2070  136.469025 -4.584036    -6.136751        1.552715
2071  136.371567 -4.208459    -5.751093        1.542634
2072  132.324295 -4.189103    -5.438695        1.249592
2073  133.247467 -4.052555    -5.161467        1.108912
2074  129.481094 -4.199842    -4.969142        0.769300


## Bollinger Bands
- Volatility measure
- Middle Band = MA20
- Upper Band = MA20 + (2 × StdDev)
- Lower Band = MA20 - (2 × StdDev)
- Narrow bands = Low volatility
- Large bands = High volatility

In [ ]:
def calculate_bollinger_bands(data, window=20, num_std=2):
    """Calculate Bollinger Bands for a given DataFrame.
    
    Args:
        data (pd.Series): Series containing 'close' prices.
        window (int): The period for the moving average. Default is 20.
        num_std (int): Number of standard deviations for the bands. Default is 2.
    
    Returns:
        Tuple of (middle_band, upper_band, lower_band, bandwidth)
    """

    # Calculate the moving average (middle band)
    middle_band = data.rolling(window=window).mean()

    # Calculate the rolling standard deviation
    rolling_std = data.rolling(window=window).std()

    # Calculate upper and lower bands
    upper_band = middle_band + (rolling_std * num_std)
    lower_band = middle_band - (rolling_std * num_std)

    # Bandwidth = (upper_band - lower_band) / middle_band
    bandwidth = (upper_band - lower_band) / middle_band

    return middle_band, upper_band, lower_band, bandwidth

df['bb_middle'], df['bb_upper'], df['bb_lower'], df['bb_bandwidth'] = calculate_bollinger_bands(df['close'])


✅ Bollinger Bands calculated!

📊 Preview with Bollinger Bands:
           close    bb_lower   bb_middle    bb_upper  bb_bandwidth
2065  133.322998  125.617365  135.832317  146.047268      0.150405
2066  132.348450  125.280465  135.586452  145.892440      0.152021
2067  132.093567  125.448760  135.648717  145.848675      0.150388
2068  133.278229  125.315096  135.276910  145.238723      0.147280
2069  137.901611  125.323293  135.333643  145.343992      0.147936
2070  136.469025  125.496643  135.481236  145.465829      0.147394
2071  136.371567  126.443578  135.874789  145.306001      0.138822
2072  132.324295  127.351245  136.113444  144.875643      0.128748
2073  133.247467  127.738841  136.240564  144.742287      0.124805
2074  129.481094  126.845603  135.796051  144.746500      0.131822


## OHLC Features
- Candlestick patterns
- Intraday volatility
- Price momentum

In [ ]:
# OHLC Features
df['price_range'] = df['high'] - df['low']
df['price_change'] = df['close'] - df['open']
df['price_change_pct'] = (df['price_change'] / df['open']) * 100

# Body size and ratio
df['body_size'] = abs(df['close'] - df['open'])
df['body_ratio'] = df['body_size'] / df['price_range'].replace(0, np.nan)

# Shadows
df['upper_shadow'] = df['high'] - df[['close', 'open']].max(axis=1)
df['lower_shadow'] = df[['close', 'open']].min(axis=1)

# Bullish/Bearish
df['is_bullish'] = np.where(df['close'] > df['open'], 1, 0)


df[['date', 'price_range', 'body_ratio', 'is_bullish']].tail(10)

✅ OHLC features calculated!

📊 Preview:


,date,price_range,body_ratio,is_bullish
2065,2025-12-05,9.244583,0.615601,0
2066,2025-12-06,2.563263,0.380734,0
2067,2025-12-07,7.947571,0.032249,0
2068,2025-12-08,7.532700,0.156947,1
2069,2025-12-09,12.872055,0.359172,1
2070,2025-12-10,6.579483,0.218417,0
2071,2025-12-11,8.294052,0.011116,0
2072,2025-12-12,9.307449,0.434667,0
2073,2025-12-13,2.259323,0.408720,1
2074,2025-12-14,4.689331,0.803212,0


## ATR (Average True Range)
- Measures market volatility
- High ATR = High volatility
- Used for position sizing

In [ ]:
def calculate_atr(data, window=14):
    """Calculate the Average True Range (ATR) for a given DataFrame.
    
    Args:
        data (pd.DataFrame): DataFrame containing 'high', 'low', and 'close' prices.
        window (int): The period over which to calculate ATR. Default is 14.
    Returns:
        pd.Series: A pandas Series containing the ATR values.
    """

    high_low = data['high'] - data['low']
    high_close = np.abs(data['high'] - data['close'].shift())
    low_close = np.abs(data['low'] - data['close'].shift())

    true_range = pd.concat([high_low, high_close, low_close], axis=1).max(axis=1)

    atr = true_range.rolling(window=window).mean()

    return atr

df['atr_14'] = calculate_atr(df, window=14)

df[['close', 'atr_14', 'price_range']].tail(10)

✅ ATR calculated!

📊 Preview:


,close,atr_14,price_range
2065,133.322998,7.626486,9.244583
2066,132.348450,7.506790,2.563263
2067,132.093567,7.661917,7.947571
2068,133.278229,7.418368,7.532700
2069,137.901611,7.849336,12.872055
2070,136.469025,7.687972,6.579483
2071,136.371567,8.032015,8.294052
2072,132.324295,8.195491,9.307449
2073,133.247467,8.159363,2.259323
2074,129.481094,8.039151,4.689331


## Volume Features
- Volume momentum and flow
- OBV (On-Balance Volume)
- Volume-Price Trend

In [ ]:
# Volume momentum
df['volume_change_pct'] = df['volume'].pct_change() * 100
df['volume_ratio'] = df['volume'] / df['volume_7d_ma']
df['volume_ratio'] = df['volume_ratio'].replace([np.inf, -np.inf], 1).fillna(1)

# OBV
df['obv'] = (np.sign(df['close'].diff()) * df['volume']).fillna(0).cumsum()

# Volume-Price Trend
df['vpt'] = ((df['close'] - df['close'].shift()) / df['close'].shift()) * df['volume'].fillna(0).cumsum()

df[['volume', 'volume_ratio', 'obv']].tail(10)

✅ Volume features calculated!

📊 Preview:


,volume,volume_ratio,obv
2065,4557254539,0.978268,3.249015e+10
2066,2061286212,0.447517,3.042887e+10
2067,3824892147,0.812732,2.660398e+10
2068,4710407082,1.048698,3.131438e+10
2069,5869893316,1.319589,3.718428e+10
2070,5949930252,1.331571,3.123435e+10
2071,5833676784,1.244713,2.540067e+10
2072,5282060650,1.102656,2.011861e+10
2073,2024059188,0.423002,2.214267e+10
2074,2489001258,0.541777,1.965367e+10


## Temporal Features
- Day of the week, month, quarter
- Market patterns (weekends, month-end)

In [ ]:
df['day_of_week'] = df['date'].dt.dayofweek
df['month'] = df['date'].dt.month
df['quarter'] = df['date'].dt.quarter
df['day_of_month'] = df['date'].dt.day

df['is_weekend'] = np.where(df['day_of_week'] >= 5, 1, 0)
df['is_month_start'] = np.where(df['day_of_month'] <= 1, 1, 0)
df['is_month_end'] = np.where(df['day_of_month'] >=25, 1, 0)

df[['date', 'day_of_week', 'is_weekend', 'month']].tail(10)

✅ Temporal features calculated!

📊 Preview:


,date,day_of_week,is_weekend,month
2065,2025-12-05,4,0,12
2066,2025-12-06,5,1,12
2067,2025-12-07,6,1,12
2068,2025-12-08,0,0,12
2069,2025-12-09,1,0,12
2070,2025-12-10,2,0,12
2071,2025-12-11,3,0,12
2072,2025-12-12,4,0,12
2073,2025-12-13,5,1,12
2074,2025-12-14,6,1,12


## Data Cleaning & Validation
- Remove rows with NaN values
- Check data quality

In [13]:
# Check for NaN values
print("🔍 NaN values per column:")
print(df.isnull().sum())

# Remove rows with NaN (mainly from rolling windows)
df_clean = df.dropna()

print(f"\n📊 Original rows: {len(df)}")
print(f"📊 After cleaning: {len(df_clean)}")
print(f"📊 Removed: {len(df) - len(df_clean)} rows")

# Display final dataset info
print(f"\n✅ Final dataset: {len(df_clean)} rows × {len(df_clean.columns)} columns")
print(f"📅 From {df_clean['date'].min()} to {df_clean['date'].max()}")

print("\n📋 All columns:")
print(df_clean.columns.tolist())

🔍 NaN values per column:
date                   0
close                  0
high                   0
low                    0
open                   0
volume                 0
close_7d_ma            6
close_30d_ma          29
close_50d_ma          49
close_200d_ma        199
volume_7d_ma           6
volume_30d_ma         29
ma_7_30_cross          0
ma_50_200_cross        0
rsi_14                13
macd                   0
macd_signal            0
macd_histogram         0
bb_middle             19
bb_upper              19
bb_lower              19
bb_bandwidth          19
price_range            0
price_change           0
price_change_pct       0
body_size              0
body_ratio             0
upper_shadow           0
lower_shadow           0
is_bullish             0
atr_14                13
volume_change_pct      1
volume_ratio           0
obv                    0
vpt                    1
day_of_week            0
month                  0
quarter                0
day_of_month           0


## Upload to Hopsworks Feature Store
- Connect to Hopsworks
- Create/get feature group
- Insert features

In [ ]:
load_dotenv(dotenv_path= "../.env")
project = hopsworks.login(api_key_value=os.getenv("HOPSWORKS_API_KEY"))
fs = project.get_feature_store()

print("Connected to Hopsworks Feature Store")

df_h = df_clean.copy()

df_h['timestamp'] = ((df_h['date'] - pd.Timestamp("1970-01-01")) // pd.Timedelta('1s')).dt.datetime

cols = cols = ['timestamp'] + [col for col in df_h.columns if col not in ['timestamp', 'date']]
df_h = df_h[cols]

print(f"\nDataset ready for upload:")
print(f"   Rows: {len(df_h)}")
print(f"   Columns: {len(df_h.columns)}")
print(f"   Date range: {df_clean['date'].min()} to {df_clean['date'].max()}")

print("Creating/getting feature group...")

solana_fg = fs.get_or_create_feature_group(
    name="solana_crypto_features",
    version=1,
    description="Features for Solana (SOL-USD) cryptocurrency",
    primary_key=["timestamp"],
    event_time="timestamp",
    online_enabled=True
)

solana_fg.insert(df_h, write_options={"wait_for_job": False})
print("Feature Group ready: 'solana_daily_features' v1")

# Upload les données (backfill)
print("\nUploading data to Hopsworks... (this may take a minute)")

2025-12-15 17:32:58,006 INFO: Closing external client and cleaning up certificates.
2025-12-15 17:32:58,008 INFO: Connection closed.
2025-12-15 17:32:58,014 INFO: Initializing external client
2025-12-15 17:32:58,015 INFO: Base URL: https://c.app.hopsworks.ai:443




To ensure compatibility please install the latest bug fix release matching the minor version of your backend (4.2) by running 'pip install hopsworks==4.2.*'


2025-12-15 17:32:59,923 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1279131
✅ Connected to Hopsworks Feature Store

📊 Dataset ready for upload:
   Rows: 1876
   Columns: 42
   Date range: 2020-10-26 00:00:00 to 2025-12-14 00:00:00
Creating/getting feature group...
Feature Group created successfully, explore it at 
https://c.app.hopsworks.ai:443/p/1279131/fs/1265740/fg/1867085


Uploading Dataframe: 100.00% |██████████| Rows 1876/1876 | Elapsed Time: 00:01 | Remaining Time: 00:00


Launching job: solana_crypto_features_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1279131/jobs/named/solana_crypto_features_1_offline_fg_materialization/executions
✅ Feature Group ready: 'solana_daily_features' v1

📤 Uploading data to Hopsworks... (this may take a minute)
